# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [4]:
!python -m pip install semantic-kernel==0.2.7.dev0

In [17]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()


useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

Let's define a prompt outlining a dialogue chat bot.

In [6]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [7]:
chat_function = kernel.create_semantic_function(sk_prompt, "ChatBot", max_tokens=2000, temperature=0.7, top_p=0.5)

Initialize your context

In [8]:
context = kernel.create_new_context()
context["history"] = ""

Chat with the Bot

In [9]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

 Hi there! What kind of books are you looking for? Fiction, non-fiction, romance, fantasy, etc.?


Update the history with the output

In [10]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot:  Hi there! What kind of books are you looking for? Fiction, non-fiction, romance, fantasy, etc.?



Keep Chatting!

In [11]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [12]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?


ChatBot:  How about Thucydides' History of the Peloponnesian War? It's a classic and provides a great overview of the events leading up to the war and its aftermath.


In [13]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?


ChatBot:  Thucydides' History of the Peloponnesian War is an account of the war between Athens and Sparta in the 5th century BC. It covers the events leading up to the war, the war itself, and its aftermath. It is considered one of the earliest works of history and is still studied today.


In [14]:
await chat("if I read that book, what exactly will I learn about Greece history?")

User: if I read that book, what exactly will I learn about Greece history?


ChatBot:  By reading Thucydides' History of the Peloponnesian War, you will learn about the causes of the war, the strategies used by both sides, and the consequences of the war for Greece. You will also gain insight into the political and social dynamics of the time, as well as the culture and values of the ancient Greeks.


In [15]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot:  Sure! Other books you could read about the Peloponnesian War include: Xenophon's Hellenica, Plutarch's Lives, Herodotus' Histories, and Thucydides' The Peloponnesian War. You could also read works by Aristotle, Plato, and other ancient Greek authors for more insight into the culture and values of the time.
